In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from binance_utils import init
from trade_utils import get_data
from machine_learning_utils import get_kmeans_clusters
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

symbol = 'BTCUSDT'
start = '2022-01-01'
end = ''
strategy_interval = '1D'

client = init()
data = get_data(client, symbol, strategy_interval)
#limit by date
#data = data[start:end]
#limit by number of candles
data = data[-60:]

def plot_all(levels, df, title):
  fig = go.Figure(data=go.Candlestick(x=df.index,
                open=df['OpenPrice'],
                high=df['HighPrice'],
                low=df['LowPrice'],
                close=df['ClosePrice']))

  last_price = df['ClosePrice'][-1]
  for level in levels:
    fig.add_annotation(x=df.index[-1], 
                       y=level[1], 
                       text='{:0.8f}'.format(level[1]),
                       yanchor='bottom',
                       font=dict(size=12, color="purple", family="Sans Serif"), 
                       align="left")

    line_color = 'red'
    if (last_price < level[1]):
      line_color = 'green'

    fig.add_shape(type="line", 
                  x0=df.index[level[0]], y0=level[1], 
                  x1=max(df.index), y1=level[1],
                  line_width=2, 
                  line_dash="dash", 
                  line_color=line_color)

  fig.update_layout(title=title, xaxis_rangeslider_visible=False, width=1400, height=800)
  fig.show()

In [ ]:
from support_resistance_utils import get_kmeans_levels, get_fractal_levels, get_window_shifting_levels, get_pivot_point_levels

plot_all(get_kmeans_levels(data), data, 'Support Resistance - Kmeans')
plot_all(get_fractal_levels(data), data, 'Support Resistance - Fractal')
plot_all(get_window_shifting_levels(data), data, 'Support Resistance - Shifting')
plot_all(get_pivot_point_levels(data), data, 'Support Resistance - Pivot Point')

In [ ]:
"""
Based on:
https://medium.com/@judopro/using-machine-learning-to-programmatically-determine-stock-support-and-resistance-levels-9bb70777cf8e
https://realpython.com/k-means-clustering-python/
"""

lows = pd.DataFrame(data=data, index=data.index, columns=['LowPrice'])
highs = pd.DataFrame(data=data, index=data.index, columns=['HighPrice'])

low_clusters, lwcss = get_kmeans_clusters(lows)
low_centers = low_clusters.cluster_centers_
low_centers = np.sort(low_centers, axis=0)

k = low_centers.size
plt.plot(range(1, 11), lwcss)
plt.scatter(range(1, 11), lwcss)
plt.scatter(k, lwcss[k], marker='X', s = 300, c = 'r')
plt.title('The Elbow Method - Support')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

high_clusters, hwcss = get_kmeans_clusters(highs)
high_centers = high_clusters.cluster_centers_
high_centers = np.sort(high_centers, axis=0)

k = high_centers.size
plt.plot(range(1, 11), hwcss)
plt.scatter(range(1, 11), hwcss)
plt.scatter(k, hwcss[k], marker='X', s = 300, c = 'r')
plt.title('The Elbow Method - Resistance')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
from support_resistance_utils import get_all_pivot_point_levels

pivot_point, support_l1, support_l2, support_l3, resistance_l1, resistance_l2, resistance_l3 = get_all_pivot_point_levels(data)

num_candles = 30
fig = go.Figure(data=go.Candlestick(x=data.index[-num_candles:],
                open=data['OpenPrice'][-num_candles:],
                high=data['HighPrice'][-num_candles:],
                low=data['LowPrice'][-num_candles:],
                close=data['ClosePrice'][-num_candles:],
                showlegend=False))

fig.add_hline(y=pivot_point[-1], 
              line_dash='dash', 
              line_color='blue', 
              opacity=0.3,
              exclude_empty_subplots=True)

# Support level 1
fig.add_hline(y=support_l1[-1], 
              line_dash='dash', 
              line_color='red', 
              opacity=0.3,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=support_l1.index[-num_candles:], 
                         y=support_l1[-num_candles:],
                         line_color='red',
                         opacity=0.3,
                         name='support 1',
                         mode='lines'))

# Support Level 2
fig.add_hline(y=support_l2[-1], 
              line_dash='dash', 
              line_color='red', 
              opacity=0.5,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=support_l2.index[-num_candles:], 
                         y=support_l2[-num_candles:],
                         line_color='red',
                         opacity=0.5,
                         name='support 2',
                         mode='lines'))

# Support Level 3
fig.add_hline(y=support_l3[-1], 
              line_dash='dash', 
              line_color='red', 
              opacity=1,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=support_l3.index[-num_candles:], 
                         y=support_l3[-num_candles:],
                         line_color='red',
                         opacity=1,
                         name='support 3',
                         mode='lines'))

# Resistance level 1
fig.add_hline(y=resistance_l1[-1], 
              line_dash='dash', 
              line_color='green',
              opacity=0.3,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=resistance_l1.index[-num_candles:], 
                         y=resistance_l1[-num_candles:],
                         line_color='green',
                         opacity=0.3,
                         name='resistance 1',
                         mode='lines'))

# Resistance level 2
fig.add_hline(y=resistance_l2[-1], 
              line_dash='dash', 
              line_color='green',
              opacity=0.5,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=resistance_l2.index[-num_candles:], 
                         y=resistance_l2[-num_candles:],
                         line_color='green',
                         opacity=0.5,
                         name='resistance 2',
                         mode='lines'))

# Resistance level 3
fig.add_hline(y=resistance_l3[-1], 
              line_dash='dash', 
              line_color='green',
              opacity=1,
              exclude_empty_subplots=True)

fig.add_trace(go.Scatter(x=resistance_l3.index[-num_candles:], 
                         y=resistance_l3[-num_candles:],
                         line_color='green',
                         opacity=1,
                         name='resistance 3',
                         mode='lines'))

fig.update_layout(title='Support - Resistance levels', xaxis_rangeslider_visible=False, width=1400, height=800)
fig.show()

In [ ]:
#levels = get_fractal_levels(data)
#if (has_breakout(levels[-5:], data.iloc[-2], data.iloc[-1])):
#  print('Breakout...')

# get the coin list
# for coin get prices, apply methods